In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import json
import glob, os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats

%matplotlib notebook
plt.rcParams.update({'font.size': 10})

In [2]:
def define_plot_pos(ax):
    ax.set_xlabel('x (m)')
    ax.set_ylabel('y (m)')
    ax.set_zlabel('z (m)')
    ax.set_xlim(-4,4)
    ax.set_ylim(-4,4)
    ax.set_zlim(0,6)
    plt.legend()
    plt.show()

In [3]:
def define_plot_col(ax):
    plt.xlabel('Time (s)')
    plt.ylabel('Collisions (bool)')
    plt.legend()
    plt.show()

# Plots for the haptics experiment

In [13]:
def dfFromJSON(file):
    with open(file) as f:
        temp = []
        for line in f:
            temp.append(json.loads(line))
            
    df = pd.io.json.json_normalize(temp[0]['allLogs'])
    df.columns = df.columns.map(lambda x: x.split("*.")[-1])
    return df

In [14]:
def appendToLogs(file, log, _type):
    df = dfFromJSON(file)
    log[_type].append(df)

# Approach wall (X)

In [16]:
path_to_folder = "./Haptics/Haptics_X/"
subjects = glob.glob(path_to_folder + "*/")

logs = {}
for curr in subjects:
    logs[curr] = []
    
subjects

['./Haptics/Haptics_X\\sub2_Controller\\',
 './Haptics/Haptics_X\\sub2_MotionCapture\\',
 './Haptics/Haptics_X\\sub3_MotionCapture\\',
 './Haptics/Haptics_X\\sub4_Controller\\',
 './Haptics/Haptics_X\\sub5_Controller\\',
 './Haptics/Haptics_X\\sub5_MotionCapture\\',
 './Haptics/Haptics_X\\sub6_Controller\\',
 './Haptics/Haptics_X\\sub6_MotionCapture\\']

In [ ]:
# # used to remove outliers

# subjects.remove(subjects[5])
# subjects.remove(subjects[4])

# logs = {}
# for curr in subjects:
#     logs[curr] = []
    
# subjects

In [17]:
EXP_N = 5

for subject in subjects:
    print(subject)
    files = glob.glob(subject+"*.json")
    files.sort()
    
    if len(files) != EXP_N:
        print (len(files))
        print('wrong number of experiments!')
        break
    for file in files:
        logs[subject].append(dfFromJSON(file))

./Haptics/Haptics_X\sub2_Controller\
./Haptics/Haptics_X\sub2_MotionCapture\
./Haptics/Haptics_X\sub3_MotionCapture\
./Haptics/Haptics_X\sub4_Controller\
./Haptics/Haptics_X\sub5_Controller\
./Haptics/Haptics_X\sub5_MotionCapture\
./Haptics/Haptics_X\sub6_Controller\
./Haptics/Haptics_X\sub6_MotionCapture\


In [19]:
fields = ['motion', 'remote']

sub_remote = [x for x in subjects if 'Controller' in x]
sub_motion = [x for x in subjects if 'Motion' in x]

logs['motion'] = [logs[x] for x in sub_motion]
logs['remote'] = [logs[x] for x in sub_remote]

In [23]:
logs[fields[0]][0]

[      absoluteTime  differentialTime  desiredYawRate  clutch  frontObstacle  \
 0        12.912371          0.012642       -0.000006   False       2.168453   
 1        12.924926          0.012555       -0.000006   False       2.168453   
 2        12.937561          0.012635       -0.000006   False       2.168453   
 3        12.949740          0.012179       -0.000006   False       2.168453   
 4        12.962297          0.012558       -0.000071   False       2.164315   
 ...            ...               ...             ...     ...            ...   
 2848     48.512901          0.012282        0.000013    True       0.282374   
 2849     48.525520          0.012616        0.000013    True       0.282374   
 2850     48.538235          0.012716        0.000013    True       0.282374   
 2851     48.550556          0.012320        0.000014    True       0.282417   
 2852     48.563122          0.012566        0.000014    True       0.282417   
 
       backObstacle  leftObstacle  rig

In [27]:
var = {}

for i in fields:
    var[i] = []
    for j in logs[i]:
        var[i].append(find_max_x(j))
    print(i)
    print(var[i])

    var['means'].append(np.mean(var[i]))
    var['stds'].append(np.std(var[i]))

    print(var['means'][-1])
    print(var['stds'][-1])

    print()

TypeError: list indices must be integers or slices, not str

In [30]:
len(logs['motion'])

4

In [ ]:
def runBasicAnalysis(f):
    
    var = {}

    var['means'] = []
    var['stds'] = []

    for i in fields:
        var[i] = []
        for j in logs[i]:
            var[i].append(f(j))
        print(i)
        print(var[i])

        var['means'].append(np.mean(var[i]))
        var['stds'].append(np.std(var[i]))

        print(var['means'][-1])
        print(var['stds'][-1])

        print()
    for i in fields:
        for j in fields:
            t, p = stats.ttest_ind(var[i],var[j])

            print (i,j)
            print (p)
            print()
            
    var['diff_mot'] = 1-np.array(var['last_motion'])/np.array(var['first_motion'])
    var['diff_rem'] = 1-np.array(var['last_remote'])/np.array(var['first_remote'])

    # var['diff_mot'] = np.array(var['last_motion'])-np.array(var['first_motion'])
    # var['diff_rem'] = np.array(var['last_remote'])-np.array(var['first_remote'])


    print('motion ratio')
    print(var['diff_mot'])
    print(np.mean(var['diff_mot']))
    print(np.std(var['diff_mot']))
    print()
    print('remote ratio')
    print(var['diff_rem'])
    print(np.mean(var['diff_rem']))
    print(np.std(var['diff_rem']))

    t, p = stats.ttest_ind(var['diff_mot'], var['diff_rem'])

    print()
    print('p = ', p)
            
    return var

# Plot Trajectories

In [ ]:
fig = plt.figure(figsize=(6,6))

plt.rcParams.update({'font.size': 6})

for idx,i in enumerate(fields):
    ax = fig.add_subplot(221+idx, projection='3d')
    for idx1,log in enumerate(logs[i]):
        ax.scatter(log['dronePosition.x'],log['dronePosition.z'],log['dronePosition.y'], s = 1, label=fields[idx] + ' try subject ' + str(idx1))
    define_plot_pos(ax)

# Collisions

In [26]:
def find_max_x(df):
    
    n = max(df['dronePosition.x'])
    
    return n

In [ ]:
coll = runBasicAnalysis(compColl)

# Time analysis

In [ ]:
def compTime(df):
    return df['absoluteTime'].iloc[-1] - df['absoluteTime'].iloc[0]

In [ ]:
time = runBasicAnalysis(compTime)

In [ ]:
size = 6

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(121)
ax.bar([0, 1, 2, 3],
   time['means'],
   yerr=time['stds'])

plt.xticks(range(4), fields, rotation = 10)

ax = fig.add_subplot(122)
ax.bar([0, 1],
   [np.mean(time_diff_mot), np.mean(time_diff_rem)],
   yerr=[np.std(time_diff_mot), np.std(time_diff_rem)])

# Distance Analysis

In [ ]:
def compDist(df):
    x = df['dronePosition.x'].values
    y = df['dronePosition.y'].values
    z = df['dronePosition.z'].values

    dx = np.diff(x)
    dy = np.diff(y)
    dz = np.diff(z)

    dd = np.sqrt(dx**2+dy**2+dz**2)
    d = np.sum(dd)

    return d

In [ ]:
dist = runBasicAnalysis(compDist)

In [ ]:
size = 6

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(121)
ax.bar([0, 1, 2, 3],
   dist['means'],
   yerr=dist['stds'])

plt.xticks(range(4), fields, rotation = 10)

ax = fig.add_subplot(122)
ax.bar([0, 1],
   [np.mean(dist['diff_mot']), np.mean(dist['diff_rem'])],
   yerr=[np.std(dist['diff_mot']), np.std(dist['diff_rem'])])

# Energy Analysis

In [ ]:
def compEnergy(df):
    df['controlInput'] = np.abs(np.linalg.norm(np.array([df['controlSpeed.x'],df['controlSpeed.y'],df['controlSpeed.z']]), axis=0))#*df['differentialTime']
    return df['controlInput'].sum()/df.shape[1]

In [ ]:
energy = runBasicAnalysis(compEnergy)

In [ ]:
size = 6

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(121)
ax.bar([0, 1, 2, 3],
   energy['means'],
   yerr=energy['stds'])

plt.xticks(range(4), fields, rotation = 10)

ax = fig.add_subplot(122)
ax.bar([0, 1],
   [np.mean(energy['diff_mot']), np.mean(energy['diff_rem'])],
   yerr=[np.std(energy['diff_mot']), np.std(energy['diff_rem'])])

In [ ]:
def compDEnergy(df):
    dEnergy = np.abs(np.diff(df['controlInput']))
    return np.sum(dEnergy)

In [ ]:
dEnergy = runBasicAnalysis(compDEnergy)

In [ ]:
size = 6

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(121)
ax.bar([0, 1, 2, 3],
   dEnergy['means'],
   yerr=dEnergy['stds'])

plt.xticks(range(4), fields, rotation = 10)

ax = fig.add_subplot(122)
ax.bar([0, 1],
   [np.mean(dEnergy['diff_mot']), np.mean(dEnergy['diff_rem'])],
   yerr=[np.std(dEnergy['diff_mot']), np.std(dEnergy['diff_rem'])])

# Analyze collisions for learning curve experiment

In [ ]:
av_collisions = {"Controller_First" : 0,
                      "Controller_Last" : 0,
                      "MotionCapture_First" : 0,
                      "MotionCapture_Last" : 0}

In [ ]:
# sum number of collisions per type
for setup in logs:
    print('processing '+setup)
    temp = 0
    for df in logs[setup]:
        temp += np.sum(np.diff(df['collision'])==1)
    av_collisions[setup] = temp / len(logs[setup])

In [ ]:
fig = plt.figure()
plt.bar(av_collisions.keys(), av_collisions.values(), color='g')
plt.ylabel('Mean number of collisions per subject')
plt.show()